In [1]:
import psycopg2
from dotenv import load_dotenv
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os


/home/shiv/miniconda3/envs/gemini/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


# Initialize GoogleGenerativeAIEmbedding
model = GoogleGenerativeAIEmbeddings(
        model='models/embedding-001',
    )


I0000 00:00:1726734942.160413  393897 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [3]:

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname="vector_db",
    user="postgres",
    password="great",
    host="localhost",
    port="5432"
)

# Create a cursor object
cursor = connection.cursor()

# Function to find similar sentences
def find_similar_sentences(query_sentence, top_n=1):
    # Generate the embedding for the query sentence
    query_embedding = model.embed_documents([query_sentence])[0]  # Get the embedding as a list
    
    # Convert the embedding to a format PostgreSQL understands (e.g., array)
    query_embedding_str = ','.join(map(str, query_embedding))
    
    # Write the SQL query to find similar sentences
    sql_query = f"""
    SELECT id, collection_id, embedding <=> ARRAY[{query_embedding_str}]::vector  AS distance
    FROM langchain_pg_embedding
    ORDER BY embedding <=> ARRAY[{query_embedding_str}]
    LIMIT {top_n};
    """
    
    # Execute the SQL query
    cursor.execute(sql_query)
    
    # Fetch the results
    results = cursor.fetchall()
    
    # Display the similar sentences
    for result in results:
        print(f"Sentence ID: {result[0]}, Sentence: {result[1]}, Distance: {result[2]}")

    # Close the cursor and connection
    cursor.close()
    connection.close()




In [5]:
# Example usage
query_sentence = "Find the most similar sentence to this."
find_similar_sentences(query_sentence)



InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [6]:
import json

# Function to find similar sentences
def find_similar_sentences(query_sentence, top_n=1):

    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
    dbname="vector_db",
    user="postgres",
    password="great",
    host="localhost",
    port="5432"
)

    # Create a cursor object
    cursor = connection.cursor()

    # Generate the embedding for the query sentence
    query_embedding = model.embed_documents([query_sentence])[0]  # Get the embedding as a list
    
    # Convert the embedding to a format PostgreSQL understands (e.g., array)
    query_embedding_str = ','.join(map(str, query_embedding))
    
    # Write the SQL query to find similar sentences, with explicit vector casting
    sql_query = f"""
    SELECT id, collection_id, document, embedding <=> '[{query_embedding_str}]'::vector AS distance
    FROM langchain_pg_embedding
    ORDER BY embedding <=> '[{query_embedding_str}]'::vector
    LIMIT {top_n};
    """
    
    # Execute the SQL query
    cursor.execute(sql_query)
    
    # Fetch the results
    results = cursor.fetchall()
    
    # Create a list of dictionaries to store the result
    similar_sentences = []
    
    for result in results:
        similar_sentences.append({
            "sentence_id": result[0],
            "collection_id": result[1],
            "sentence": result[2],
            "distance": result[3]
        })

    # Convert the list of dictionaries to JSON
    json_response = json.dumps(similar_sentences)

    # Close the cursor and connection
    cursor.close()
    connection.close()

    return json_response



In [7]:
# Example usage
query_sentence = "If you want to achieve greatness stop asking for"
res = find_similar_sentences(query_sentence)

In [8]:
res

'[{"sentence_id": "5d4f5fd0-1c0b-4bf2-a92a-3621509cc286", "collection_id": "dc497641-9d03-4142-9ed1-156a80881bb7", "sentence": "If you want to achieve greatness stop asking for permission. ~Anonymous", "distance": 0.05026439880878031}]'

In [9]:
import json
json_res = json.dumps(res)

In [38]:
json_res

'[{"sentence_id": "5d4f5fd0-1c0b-4bf2-a92a-3621509cc286", "collection_id": "dc497641-9d03-4142-9ed1-156a80881bb7", "sentence": "If you want to achieve greatness stop asking for permission. ~Anonymous", "distance": 0.05026439880878031}]'

In [39]:
res

[{'sentence_id': '5d4f5fd0-1c0b-4bf2-a92a-3621509cc286',
  'collection_id': 'dc497641-9d03-4142-9ed1-156a80881bb7',
  'sentence': 'If you want to achieve greatness stop asking for permission. ~Anonymous',
  'distance': 0.05026439880878031}]

In [10]:
# Sample sentence to generate the embedding
sample_sentence = "This is a test sentence to find embedding dimension."

# Generate the embedding for the sample sentence
embedding = model.embed_documents([sample_sentence])[0]

# Get the dimension (length) of the embedding
embedding_dimension = len(embedding)

print(f"The embedding dimension is: {embedding_dimension}")


The embedding dimension is: 768
